In [1]:
!pip install pyctcdecode
!pip install https://github.com/kpu/kenlm/archive/master.zip

  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Preparing metadata (setup.py) ... done


In [2]:
import re
def lowerjv(s):
    return re.sub(" +", " ", re.sub("[^a-z \n]"," ", s.lower().replace("j", "i").replace("v", "u"))).strip()

In [3]:
from pathlib import Path
Path("nd-latin-lowerjv.txt").write_text(lowerjv(Path("nd-latin.txt").read_text()))

18694697

In [ ]:
# Then a miracle occurs

In [4]:
with open("kenlm-nd-latin.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)


In [5]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("lsb/wav2vec2-base-pemlsb-la")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [7]:
from pyctcdecode import build_ctcdecoder
import kenlm

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)


Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


In [8]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)


In [14]:
import torchaudio
import torch
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "lsb/wav2vec2-base-pemlsb-la", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

for f in ["./vivamus.mp3", "./vae11.mp3", "./poetaexmachina-mp3-recitations/mp3/1100000", "./poetaexmachina-mp3-recitations/mp3/1010000"]:
    soundfile = torchaudio.transforms.Resample(22050, 16000)(torchaudio.load(f, format="mp3")[0]).numpy()
    soundfile_input_values = processor_with_lm(soundfile, sampling_rate=16000)['input_values'][0][0]
    logits = model(torch.tensor([soundfile_input_values])).logits
    pred_ids = torch.argmax(logits, dim=-1)
    print(processor.batch_decode(pred_ids))


['uiuamus mea lesbia atque amemus rumoresque senum se ueriorum omnes unius ae stimemus assis']
['arma uirumque cano troiae qui primus ab oris']
['gloriabuntur']
['conspirauerunt']


In [15]:
processor_with_lm.save_pretrained("wav2vec2-base-pemlsb-la")

In [16]:
from huggingface_hub import Repository

repo = Repository(local_dir="wav2vec2-base-pemlsb-la")

In [17]:
repo.push_to_hub(commit_message="Upload lm-boosted decoder")

Adding files tracked by Git LFS: ['language_model/5gram_correct.arpa']. This may take a bit of time if the files are large.


Upload file language_model/5gram_correct.arpa:   0%|          | 32.0k/408M [00:00<?, ?B/s]

To https://huggingface.co/lsb/wav2vec2-base-pemlsb-la
   66874a0..3e1c71b  main -> main



'https://huggingface.co/lsb/wav2vec2-base-pemlsb-la/commit/3e1c71baad1be8cb63d758dda7fc3d901fa7be6f'